<h1 align=center>Capítulo 6: Juntando tudo: Análise semântica com spaCy</h1>
<p align=center><img src=https://miro.medium.com/max/1050/1*6WATd2e85VuqFhkg5RQonQ.jpeg width=500></p>

Esta é uma seção puramente prática. Neste capítulo, aplicaremos o que aprendemos até agora ao **Airline Travel Information System (ATIS)**, um conhecido conjunto de dados do sistema de reserva de passagens aéreas. Em primeiro lugar, conheceremos nosso conjunto de dados e faremos as estatísticas básicas. Como a primeira tarefa de **compreensão de linguagem natural (NLU)**, vamos extrair as entidades nomeadas com dois métodos diferentes, com *spaCy Matcher*, e andando na árvore de dependências.

A próxima tarefa é determinar a intenção do enunciado do usuário. Também exploraremos o reconhecimento de intenção de diferentes maneiras: extraindo os verbos e seus objetos diretos, usando listas de palavras e andando na árvore de dependência para reconhecer várias intenções. Em seguida, você combinará suas palavras-chave com sinônimos de uma lista de sinônimos para detectar semelhança semântica.

Além disso, você fará a correspondência de palavras-chave com métodos de semelhança semântica baseados em vetor de palavras. Por fim, combinaremos todas essas informações para gerar uma representação semântica para os enunciados do conjunto de dados.

No final deste capítulo, você aprenderá a processar completamente um conjunto de dados do mundo real semanticamente. Você aprenderá a extrair entidades, reconhecer *intents* e realizar cálculos de similaridade semântica. As ferramentas deste capítulo são realmente o que você construirá para um pipeline de **processamento de linguagem natural (NLP)** do mundo real, incluindo um chatbot NLU e um aplicativo de suporte ao cliente NLU.

Neste capítulo, abordaremos os seguintes tópicos principais:
1. Extraindo entidades nomeadas
2. Usando relações de dependência para reconhecimento de intenção
3. Métodos de similaridade semântica para análise semântica
4. Juntando tudo

## Extraindo entidades nomeadas
Em muitos aplicativos de NLP, incluindo análise semântica, começamos a procurar significado em um texto examinando os tipos de entidade e colocando um componente de extração de entidade em nossos pipelines de NLP. **As entidades nomeadas** desempenham um papel fundamental na compreensão do significado do texto do usuário.

Também iniciaremos um pipeline de análise semântica extraindo as entidades nomeadas de nosso corpus. Para entender que tipo de entidades queremos extrair, primeiro, conheceremos o conjunto de dados ATIS.

### Conhecendo o conjunto de dados ATIS
Ao longo deste capítulo, trabalharemos com o corpus ATIS. ATIS é um conjunto de dados bem conhecido; é um dos conjuntos de dados de referência padrão para classificação de intenção. O conjunto de dados consiste em declarações de clientes que desejam reservar um voo, obter informações sobre os voos, incluindo custos de voo, destinos de voo e horários.

Não importa qual seja a tarefa da NLP, você deve sempre examinar seu corpus a olho nu. Queremos conhecer nosso corpus para integrar nossas observações de corpus em nosso código. Ao visualizar nossos dados de texto, geralmente observamos o seguinte:
* Que tipo de enunciados existem? É um corpus de texto curto ou o corpus consiste em documentos longos ou parágrafos de tamanho médio?
* Que tipo de entidades o corpus inclui? Nomes de pessoas, nomes de cidades, nomes de países, nomes de organizações e assim por diante. Quais queremos extrair?
* Como a pontuação é usada? O texto está pontuado corretamente ou nenhuma pontuação é usada?
* Como as regras gramaticais são seguidas? A capitalização está correta? Os usuários seguiram as regras gramaticais? Existem palavras incorretas?

Antes de iniciar qualquer processamento, examinaremos nosso corpus. Vamos em frente e baixar o conjunto de dados:
~~~python
https://github.com/PacktPublishing/Mastering-spaCy/blob/main/Chapter06/data/atis_intents.csv
~~~

O conjunto de dados é um arquivo CSV de duas colunas. Primeiro, obteremos alguns insights sobre as estatísticas do conjunto de dados com pandas. pandas é uma biblioteca de manipulação de dados popular que é frequentemente usada por cientistas de dados. Você pode ler mais em https://pandas.pydata.org/pandas-docs/version/0.15/tutorials.html.

1. Vamos começar lendo o arquivo CSV em Python. Usaremos o método **read_csv** dos pandas:

In [85]:
import pandas as pd
dataset = pd.read_csv('atis_intents.csv', header=None, encoding='utf8', sep=',')

A variável do conjunto de dados mantém o CSV como um objeto para nós.
2. Em seguida, chamaremos **head()** no objeto dataset. **head()** gera as primeiras 10 colunas do conjunto de dados:

In [86]:
dataset.head()

,0,1
0,atis_flight,i want to fly from boston at 838 am and arriv...
1,atis_flight,what flights are available from pittsburgh to...
2,atis_flight_time,what is the arrival time in san francisco for...
3,atis_airfare,cheapest airfare from tacoma to orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...


Como você vê, o objeto de conjunto de dados contém linhas e colunas. Na verdade, é um objeto CSV. A primeira coluna contém a intenção e a segunda coluna contém o enunciado do usuário.
3. Agora podemos imprimir alguns enunciados de exemplo:

In [87]:
for text in dataset[1].head():
	print(text)

 i want to fly from boston at 838 am and arrive in denver at 1110 in the morning
 what flights are available from pittsburgh to baltimore on thursday morning
 what is the arrival time in san francisco for the 755 am flight leaving washington
 cheapest airfare from tacoma to orlando
 round trip fares from pittsburgh to philadelphia under 1000 dollars


Como podemos ver, o primeiro usuário deseja reservar um voo; incluíam o destino, as cidades de origem e o tempo de voo. O terceiro usuário está perguntando sobre o horário de chegada de um voo específico e o quinto usuário fez uma consulta com um limite de preço. Os enunciados não são capitalizados ou pontuados. Isso ocorre porque esses enunciados são uma saída de um mecanismo de *speech-to-text*.

4. Por último, podemos ver a distribuição do número de enunciados por intenção:

In [88]:
grouped = dataset.groupby(0).size()
print(grouped)

0
atis_abbreviation                            147
atis_aircraft                                 81
atis_aircraft#atis_flight#atis_flight_no       1
atis_airfare                                 423
atis_airfare#atis_flight_time                  1
atis_airline                                 157
atis_airline#atis_flight_no                    2
atis_airport                                  20
atis_capacity                                 16
atis_cheapest                                  1
atis_city                                     19
atis_distance                                 20
atis_flight                                 3666
atis_flight#atis_airfare                      21
atis_flight_no                                12
atis_flight_time                              54
atis_ground_fare                              18
atis_ground_service                          255
atis_ground_service#atis_ground_fare           1
atis_meal                                      6
atis_quantity     

5. Após este ponto, processaremos apenas o texto do enunciado. Portanto, podemos descartar a primeira coluna. Para fazer isso, vamos fazer um pequeno truque com a ferramenta Unix awk:
~~~python
awk -F ',' '{print $2}' atis_intents.csv > atis_utterances.txt
~~~
Aqui, imprimimos a segunda coluna do arquivo CSV de entrada (onde o separador de arquivo é um **,**) e direcionamos a saída para um arquivo de texto chamado **atis_utterances.txt**. Agora que nossos enunciados estão prontos para serem processados, podemos prosseguir e extrair as entidades.

In [89]:
dataset[1].to_csv('atis_utterances.csv', header=None)

### Extraindo entidades nomeadas com o Matcher
Como já vimos, este é um conjunto de dados de voos. Portanto, esperamos ver nomes de cidades/países, nomes de aeroportos e nomes de companhias aéreas:
1. Aqui estão alguns exemplos:
~~~python
does american airlines fly from boston to san francisco
what flights go from dallas to tampa
show me the flights from montreal to chicago
what flights do you have from ontario
The users also provide the dates, times, days of the weeks they
wish to fly on. These entities include numbers, month names, day
of the week names as well as time adverbs such as next week,
today, tomorrow, next month. Let's see some example entities:
list flights from atlanta to boston leaving between 6 pm and 10
pm on august eighth
i need a flight after 6 pm on wednesday from oakland to salt
lake city
show me flights from minneapolis to seattle on july second
what flights leave after 7 pm from philadelphia to boston
~~~
2. Além disso, a intenção **atis_abbreviation** contém enunciados que são perguntas sobre algumas abreviações. As abreviações de voos podem ser códigos de tarifa (por exemplo, M = Economy), códigos de nome de companhia aérea (por exemplo, United Airlines = UA) e códigos de aeroporto (por exemplo, Aeroporto de Berlim = BER) e assim por diante. Os exemplos incluem o seguinte:
~~~python
what does the abbreviation ua mean
what does restriction ap 57 mean
explain restriction ap please
what's fare code yn
~~~
3. Vamos visualizar alguns enunciados do conjunto de dados. A captura de tela a seguir mostra as entidades com seus tipos:

In [90]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_md')

doc1 =nlp('show me the flights from montreal to chicago')
doc2 = nlp('does american airlines fly from boston to san francisco')
doc3 = nlp('show me flights from minneapolis to seattle on july second')
doc4 = nlp("what flights leave after 7 pm from philadelphia to boston")

displacy.render(doc1, style='ent')
displacy.render(doc2, style='ent')
displacy.render(doc3, style='ent')
displacy.render(doc4, style='ent')

4. Podemos ver todos os tipos de entidade e suas frequências de forma mais sistemática. O segmento de código a seguir realiza as seguintes ações:
   * Lê o arquivo de texto dos enunciados que criamos na subseção anterior de exploração do conjunto de dados.
   * Ele itera sobre cada enunciado e cria um objeto Doc.
   * Extrai entidades do objeto doc atual.
   * Atualiza a lista global de rótulos de entidades com os rótulos das entidades.
   * Finalmente calcula a frequência de cada etiqueta com um objeto contador.

Aqui está o código:

In [91]:
from collections import Counter
import spacy
nlp = spacy.load("en_core_web_md")
corpus = open("atis_utterances.csv", "r").read().split("\n")
all_ent_labels = []
for sentence in corpus:
    doc = nlp(sentence.strip())
    ents = doc.ents
    all_ent_labels += [ent.label_ for ent in ents]
c = Counter(all_ent_labels)
print(c)

Counter({'GPE': 9125, 'CARDINAL': 4144, 'DATE': 2283, 'TIME': 993, 'ORG': 420, 'ORDINAL': 218, 'NORP': 74, 'QUANTITY': 44, 'MONEY': 42, 'LOC': 17, 'PRODUCT': 16, 'FAC': 9, 'PERSON': 6, 'EVENT': 2})


Nosso palpite inicial parece correto. Os rótulos de entidade mais frequentes são **GPE** (nomes de local), **DATE**, **TIME**, **CARDINAL** e **ORGANIZATION**. Obviamente, as entidades de localização referem-se a cidades/países de destino e origem, portanto, desempenham um papel muito importante no sucesso semântico geral de nossa aplicação.

5. Primeiro vamos extrair as entidades de localização pelo spaCy Matcher procurando por um padrão do formulário de **preposição nome_local**. O código a seguir extrai entidades de localização precedidas por uma preposição:

In [92]:
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_md")
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "ADP"}, {"ENT_TYPE": "GPE"}]
matcher.add("prepositionLocation", [pattern])
doc = nlp("show me flights from denver to boston on tuesday")
matches = matcher(doc)
for mid, start, end in matches:
    print(doc[start:end])

from denver
to boston


In [93]:
# Visualizando ADP e GPE na frase.
displacy.render(doc, style='dep')

Já vimos como inicializar um objeto Matcher e adicionar padrões a ele. Ainda assim, vamos relembrar como usar um objeto Matcher para extrair as correspondências agora. Aqui está o que fizemos neste segmento de código:

a) Começamos importando o **spacy** e a classe **spacy.matcher** nas linhas 1-2.
b) Criamos um objeto pipeline de linguagem, **nlp**, na linha 3.
c) Na linha 4, inicializamos o objeto Matcher com o vocabulário da linguagem.
d) Na linha 5, criamos um padrão combinando dois tokens, uma preposição (tag **POS ADP** significa *adposição = preposição + posposição*) e uma entidade de localização (rótulo **GPE** significa uma *entidade de localização*).
e) Adicionamos este padrão ao objeto Matcher.
f) Por fim, solicitamos as correspondências em uma sentença corpus de exemplo e imprimimos as correspondências.

6. Embora as preposições **from** e **to** dominem neste conjunto de dados, os verbos sobre **leaving** e **arriving** podem ser usados com uma variedade de preposições. Aqui estão mais algumas frases de exemplo do conjunto de dados:

In [94]:
doc = nlp("i'm looking for a flight that goes from ontario to atlanta and stops in chicago")
matches = matcher(doc)
for mid, start, end in matches:
	print(doc[start:end])

from ontario
to atlanta
in chicago


In [95]:
# Visualizando ADP e GPE na frase.
displacy.render(doc, style='ent')

In [96]:
doc = nlp("what flights arrive in chicago on sunday on continental")
matches = matcher(doc)
for mid, start, end in matches:
    print(doc[start:end])

in chicago


Outra frase de exemplo do conjunto de dados contém uma abreviação em uma entidade de destino:

In [97]:
doc = nlp("yes i'd like a flight from Long Beach to St. louis by way of dallas")
matches = matcher(doc)
for mid, start, end in matches:
    print(doc[start:end])

from Long
to St.
of dallas


Nossa última frase de exemplo é novamente uma frase de pergunta:

doc = nlp("what are the evening flights flying out of dallas")
matches = matcher(doc)
for mid, start, end in matches:
    print(doc[start:end])

Aqui, vemos alguns verbos frasais, **arrive in**, bem como preposições e combinações de verbos, como **stop in** e **fly out of**. **By the way de Dallas** não inclui um verbo em tudo. O usuário indicou que deseja fazer uma parada em Dallas. **to, from, in, out e of*** são preposições comuns que são usadas em um contexto de viagem.

7. Após extrair as localizações, agora podemos extrair as informações da companhia aérea. O rótulo de entidade **ORG** significa uma organização e corresponde aos nomes das companhias aéreas em nosso conjunto de dados. O segmento de código a seguir extrai os nomes das organizações, possivelmente nomes com várias palavras:

In [98]:
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_md")
matcher = Matcher(nlp.vocab)

pattern = [{"ENT_TYPE": "ORG", "OP": "+"}]
matcher.add("AirlineName", [pattern])
doc = nlp("what is the earliest united airlines flight flying from denver")
matches = matcher(doc)
for mid,start,end in matches:
    print(doc[start:end])

united
united airlines
airlines


Aqui, extraímos as entidades cujos rótulos são **ORG**. Queríamos capturar uma ou mais ocorrências para capturar também as entidades de várias palavras, e é por isso que usamos o operador **OP: "+"**.

8. As datas e horários de extração não são muito diferentes; você pode replicar o anterior com código com **ENT_TYPE: DATE** e **ENT_TYPE: TIME**. Nós encorajamos você a tentar você mesmo. A captura de tela a seguir mostra detalhadamente a aparência das entidades de data e hora:

In [99]:
doc1 = nlp("show me all flights from atlanta to denver which leave after 5 o'clock pm the day after tomorrow")
doc2 = nlp("show me all flights from boston to pittsburgh next wednesday nigth after 6 o'clock")
doc3 = nlp('show me all the delta flights leaving or arriving at pittsburgh between 12 and 4 in the afternoon')
doc4 = nlp("show me the fligths before 11 am on august second from boston to denver on delta")

displacy.render(doc1, style='ent')
displacy.render(doc2, style='ent')
displacy.render(doc3, style='ent')
displacy.render(doc4, style='ent')

9. Em seguida, vamos extrair entidades do tipo abreviação. Extrair as entidades de abreviação é um pouco mais complicado. Primeiro, veremos como as abreviações aparecem:
~~~python
what does restriction ap 57 mean?
what does the abbreviation co mean?
what does fare code qo mean
what is the abbreviation d10
what does code y mean
what does the fare code f and fn mean
what is booking class c
~~~

Apenas uma dessas frases inclui uma entidade. A primeira frase de exemplo inclui uma entidade **AMOUNT**, que é **57**. Fora isso, as abreviações não são marcadas com nenhum tipo de entidade. Nesse caso, temos que fornecer algumas regras personalizadas para o Matcher. Vamos fazer algumas observações primeiro e depois formar um padrão Matcher:
a) Uma abreviatura pode ser dividida em duas partes – letras e dígitos.
b) A parte da letra pode ter de 1 a 2 caracteres.
c) A parte do dígito também tem 1-2 caracteres.
d) A presença de dígitos indica uma entidade abreviada.
e) A presença das seguintes palavras indica uma entidade de abreviatura: classe, código, abreviatura.
f) A etiqueta **POS** de uma abreviatura é um substantivo. Se a palavra candidata for uma palavra de 1 ou 2 letras, podemos olhar para a etiqueta **POS** e ver se é um substantivo. Essa abordagem elimina os falsos positivos, como us (pronome), me (pronome), a (determinador) e an (determinador).

10. Vamos agora colocar essas observações nos padrões do Matcher:

In [100]:
pattern1 = [{"TEXT": {"REGEX": "\w{1,2}\d{1,2}"}}]
pattern2 = [{"SHAPE": { "IN": ["x", "xx"]}}, {"SHAPE": { "IN":["d", "dd"]}}]
pattern3 = [{"TEXT": {"IN": ["class", "code", "abbrev","abbreviation"]}}, {"SHAPE": { "IN": ["x", "xx"]}}]
pattern4 = [{"POS": "NOUN", "SHAPE": { "IN": ["x", "xx"]}}]

Em seguida, criamos um objeto Matcher com os padrões que definimos:

In [101]:
matcher = Matcher(nlp.vocab)
matcher.add("abbrevEntities", [pattern1, pattern2, pattern3, pattern4])

Agora estamos prontos para alimentar nossas frases no matcher:

In [102]:
sentences = [
    'what does restriction ap 57 mean',
    'what does the abbreviation co mean',
    'what does fare code qo mean',
    'what is the abbreviation d10',
    'what does code y mean',
    'what does the fare code f and fn mean',
    'what is booking class c'
]

18. Estamos prontos para enviar nossas frases para o matcher:

In [103]:
for sent in sentences:
	doc = nlp(sent)
	matches = matcher(doc)
	for mid, start, end in matches:
		print(doc[start: end])

ap 57
57
abbreviation co
co
code qo
d10
code y
code f
class c
c


No código anterior, definimos quatro padrões:
a) O primeiro padrão corresponde a um único token, que consiste em 1-2 letras e 1-2 dígitos. Por exemplo, **d1, d10, ad1 e ad21** corresponderão a esse padrão.
b) O segundo padrão corresponde a abreviaturas de 2 tokens em que o primeiro token tem 1-2 letras e o segundo token 1-2 dígitos. As abreviaturas **ap 5, ap 57, a 5 e a 57** irão corresponder a este padrão.
c) O terceiro padrão também corresponde a dois tokens. O primeiro token é uma palavra de pista de contexto, como **class** ou **code**, e o segundo token deve ser um token de 1-2 letras. Algumas correspondências de exemplo são o **code f**, **code y** e a **class c**.
d) O quarto padrão extrai palavras curtas de 1-2 letras cuja tag **POS** é **NOUN**. Alguns exemplos de correspondências das frases anteriores são **c** e **co**.

spaCy Matcher torna a vida mais fácil para nós, permitindo-nos fazer uso de forma de token, dicas de contexto e uma tag de token **POS**. Fizemos uma extração de entidade muito bem-sucedida nesta subseção extraindo locais, nomes de companhias aéreas, datas, horários e abreviações. Na próxima subseção, aprofundaremos a sintaxe da frase e extrairemos entidades das frases onde o contexto não oferece muitas pistas.

### Usando árvores de dependência para extrair entidades
Na subseção anterior, extraímos entidades onde o contexto fornece pistas óbvias. Extrair a cidade de destino da frase a seguir é fácil. Podemos procurar o padrão **to + GPE**:
~~~python
I want to fly to Munich tomorrow
~~~
Mas suponha que o usuário forneça uma das seguintes frases:
~~~python
I'm going to a conference in Munich. I need an air ticket.
My sister's wedding will be held in Munich. I'd like to book a flight.
~~~

Aqui, a preposição **to** refere-se a **conference**, não a **Munich**, na primeira frase. Nesta frase, precisamos de um padrão como para **+ .... + GPE**. Então, temos que ter cuidado com quais palavras podem vir entre "to" e o nome da cidade, bem como quais palavras não devem vir. Por exemplo, esta frase tem um significado completamente diferente e não deve corresponder:
~~~python
I want to book a flight to my conference without stopping at Berlin.
~~~

Na segunda frase, não encontramos a presença do **to**. Aqui, como vemos a partir desses exemplos, precisamos examinar as relações sintáticas entre as palavras. No Capítulo 3, Características Linguísticas, já vimos como interpretar árvores de dependência para entender as relações entre palavras. Nesta subseção, percorreremos as árvores de dependência.

Percorrer uma árvore de dependências significa visitar os tokens em uma ordem personalizada, não necessariamente da esquerda para a direita. Normalmente, paramos de iterar na árvore de dependências quando encontramos o que estamos procurando. Novamente, uma árvore de dependência mostra as relações sintáticas entre suas palavras. Lembre-se do Capítulo 3, Características linguísticas, que as relações são representadas com setas direcionadas, conectando a cabeça e o filho de uma relação. Cada palavra em uma frase tem que envolver pelo menos uma relação. Esse fato garante que visitaremos cada palavra enquanto percorremos a frase.

> #### RECALL
> Antes de prosseguir com o código, primeiro, vamos relembrar alguns conceitos sobre árvores de dependência. ROOT é um rótulo de dependência especial e é sempre atribuído ao verbo principal da frase. spaCy mostra relações sintáticas com arcos. Um dos tokens é o pai sintático (chamado de HEAD) e o outro é dependente (chamado de CHILD). A título de exemplo, na Figura 6.3, **going** tem 3 filhos sintáticos – **I**, **m** e  **to**. De forma equivalente, o núcleo sintático de **to** está **going** (o mesmo se aplica a **I** e **m**).

Voltando aos nossos exemplos, vamos iterar as árvores de dependência de enunciados para descobrir se a preposição **to** está sintaticamente relacionada à entidade de localização, **Munich**. Antes de tudo, vamos ver a análise de dependência da nossa frase de exemplo *I'm going to a conference in Munich*  e também lembrar como é uma árvore de dependência:

In [104]:
doc = nlp("I'm going to a conference in Munich.")
displacy.render(doc, style='dep')

Não há arcos de entrada no verbo **going**, então **going** é a raiz da árvore de dependência (quando examinamos o código, veremos que o rótulo de dependência é **ROOT**). Isso deve acontecer porque **going** é o verbo principal da frase. Se seguirmos o arco à sua direita imediata, encontraremos **to**; saltando sobre os arcos à direita chegamos a **Munich**. Isso mostra que há uma relação sintática entre **to** e **Munich**.
Vamos agora iterar sobre a árvore de dependências com código. Existem duas maneiras possíveis de se conectar **to** e **Munich**:
 * Da esquerda para a direita. Partimos de **to** e tentamos chegar a Munique visitando os filhos sintáticos de "to". Essa abordagem pode não ser uma boa ideia, porque se "to" tiver mais de um filho, precisamos verificar cada filho e acompanhar todos os caminhos possíveis.
 * Direita para esquerda. Partimos de **Munich**, pulamos em sua cabeça, seguimos a cabeça da cabeça e assim por diante. Como cada palavra tem exatamente uma cabeça, é garantido que haverá apenas um caminho. Em seguida, determinamos se **to** está nesse caminho ou não.

Os segmentos de código a seguir implementam a segunda abordagem, iniciam a caminhada da árvore de dependência de **Munich** e procuram por **to**:

In [105]:
import spacy
nlp = spacy.load("en_core_web_md")

def reach_parent(source_token, dest_token):
	source_token = source_token.head
	while source_token != dest_token:
		if source_token.head == source_token:
			return None
		source_token = source_token.head
		return source_token
doc = nlp("I'm going to a conference in Munich.")
doc[-2]

Munich

In [106]:
doc[3]

to

In [107]:
doc[-1]

.

In [108]:
reach_parent(doc[-2], doc[3])

conference

In [109]:
reach_parent(doc[-1], doc[3])

Na função **reach_parent**, se aplica o seguinte:
* Partimos de um token de origem e tentamos alcançar o token de destino.
* No loop **while**, iteramos sobre o cabeçalho de cada token, começando pelo token de origem.
* O loop para quando alcançamos o token de origem ou a raiz da sentença.
* Testamos para alcançar a raiz através da linha **source_token == source_token.head**. Como o token raiz sempre se refere a si mesmo, sua cabeça é ele mesmo (lembre-se de que na árvore de dependências, não há arcos de entrada na raiz).
* Finalmente, testamos nossa função em dois casos de teste diferentes. No primeiro, a origem e o destino estão relacionados, enquanto no segundo teste não há relação, portanto a função retorna **None**.

Essa abordagem é muito diferente da abordagem bastante direta da subseção anterior. A linguagem natural é complicada e desafiadora de processar, e é importante conhecer as abordagens disponíveis e ter as ferramentas necessárias em sua caixa de ferramentas quando precisar usá-las.

Em seguida, vamos mergulhar em um assunto popular – reconhecimento de intenção com uma abordagem sintática. Vamos ver alguns pontos-chave do design de um bom componente de reconhecimento de intenção, incluindo o reconhecimento de várias intenções também.

## Usando relações de dependência para reconhecimento de intenção

Depois de extrair as entidades, queremos descobrir que tipo de intenção o usuário carrega – reservar um voo, comprar uma refeição em seu voo já reservado, cancelar seu voo e assim por diante. Se você olhar para a lista de *intents* novamente, verá que cada *intent* inclui um verbo (*to book*) e um objeto sobre o qual o verbo atua (*flight, hotel, meal*).

Nesta seção, extrairemos verbos transitivos e seus objetos diretos de enunciados. Começaremos nossa seção de reconhecimento de intenção extraindo o verbo transitivo e o objeto direto do verbo. Em seguida, exploraremos como entender a intenção de um usuário reconhecendo sinônimos de verbos e substantivos. Por fim, veremos como determinar a intenção de um usuário com métodos de similaridade semântica. Antes de passarmos para a extração de verbos transitivos e seus objetos diretos, vamos primeiro examinar rapidamente os conceitos de verbos transitivos e objetos diretos/indiretos.

### Cartilha linguística

Nesta seção, exploraremos alguns conceitos linguísticos relacionados à estrutura da frase, incluindo verbos e relações verbo-objeto. Um verbo é um componente muito importante da frase, pois indica a ação na frase. O objeto da frase é a coisa/pessoa que é afetada pela ação do verbo. Portanto, há uma conexão natural entre o verbo da sentença e os objetos. O conceito de transitividade capta as relações verbo-objeto. Um verbo transitivo é um verbo que precisa de um objeto para agir. Vejamos alguns exemplos:
~~~python
I bought flowers.
He loved his cat.
He borrowed my book.
~~~
Nestas frases de exemplo,**bought**, **loved** e **borrowed** são verbos transitivos. Na primeira frase, **bought** é o verbo transitivo e **flowers** é seu objeto, a coisa que foi comprada pelo sujeito da frase, **I**. **Loved** – **his cat** e **borrowed** – **my book** são exemplos verbo-objeto transitivo. Vamos nos concentrar na primeira frase novamente - o que acontece se apagarmos o objeto **flowers**?
~~~python
I bought
~~~
Comprou **o quê**? Sem um objeto, esta frase não tem nenhum significado. Nas frases anteriores, cada um dos objetos completa o significado do verbo. Essa é uma forma de entender se um verbo é transitivo ou não – apague o objeto e verifique se a frase permanece semanticamente intacta.

Alguns verbos são transitivos e alguns verbos são intransitivos. Um **verbo intransitivo** é o oposto de um verbo transitivo; ele não precisa de um objeto para agir. Vejamos alguns exemplos:
* Yesterday I **slept** for 8 hours.
* The cat <b>ran</b> towards me.
* When I went out, the sun was <b>shining</b>.
* Her cat <b>died</b> 3 days ago.

Em todas as frases anteriores, os verbos fazem sentido sem um objeto. Se apagarmos todas as palavras, exceto o sujeito e o objeto, essas frases ainda terão significado:
* I **slept**.
* The cat **ran**.
* The sun was **shining**.
* Her cat **died**.

Emparelhar um verbo intransitivo com um objeto não faz sentido. Você não pode correr alguém/algo, você não pode brilhar algo/alguém, e você certamente não pode morrer algo/alguém.

#### Objeto da sentença
Como comentamos anteriormente, o objeto é a coisa/pessoa que é afetada pela ação do verbo. A ação declarada pelo verbo é cometida pelo sujeito da sentença e o objeto da sentença é afetado.

Uma frase pode ser direta ou indireta. Um **objeto direto** responde às perguntas **quem? / o que?** Você pode encontrar o objeto direto perguntando ao **sujeito {verbo} o que/quem?**. Aqui estão alguns exemplos:
~~~python
I bought flowers. I bought what? - flowers
He loved his cat. He loved who? - his cat
He borrowed my book. He borrowed what? - my book
~~~
Um **objeto indireto** responde às perguntas **para quê?/para quem?/para quem?**. Vejamos alguns exemplos:
~~~python
He gave me his book. He gave his book to whom? - me
He gave his book to me. He gave his book to whom? - me
~~~

Os objetos indiretos geralmente são precedidos pelas preposições *to, for, from*, e assim por diante. Como você pode ver nesses exemplos, um objeto indireto também é um objeto e é afetado pela ação do verbo, mas seu papel na frase é um pouco diferente. Um objeto indireto às vezes é visto como o destinatário do objeto direto.

Isso é tudo que você precisa saber sobre verbos transitivos/intransitivos e objetos diretos/indiretos para digerir o material deste capítulo. Se você quiser aprender mais sobre a sintaxe das frases, leia o ótimo livro *Linguistic Fundamentals for Natural Language Processing de Emily Bender*: (https://dl.acm.org/doi/book/10.5555/2534456). Cobrimos o básico da sintaxe de sentenças, mas este ainda é um ótimo recurso para aprender sobre sintaxe em profundidade.

### Extraindo verbos transitivos e seus objetos diretos
Ao reconhecer a intenção, geralmente aplicamos estas etapas a um enunciado do usuário:
1. Dividindo a frase em tokens.
2. A análise de dependência é realizada pelo *spaCy*. Percorremos a árvore de dependências para extrair os tokens e as relações que nos interessam, que são o verbo e o objeto direto, conforme mostrado no diagrama a seguir:

In [110]:
doc = nlp('find a flight from washington to sf')
displacy.render(doc, style='dep')

Nesta frase de exemplo, o verbo transitivo é **find** e o objeto direto é **a flight**. A relação **dobj** conecta um verbo transitivo ao seu objeto direto. Se seguirmos o arco, semanticamente, veremos que o usuário deseja realizar a ação de encontrar e o objeto que deseja encontrar é um voo. Podemos mesclar **find** e **a flight** em uma única palavra, **findAflight** ou **findFlight**, que pode ser o nome dessa *intent*. Outras intenções podem ser **bookFlight**, **cancelFlight**, **bookMeal** e assim por diante.

Vamos extrair o verbo e o objeto direto de forma mais sistemática. Primeiro, identificaremos o objeto direto procurando o rótulo **dobj** na frase. Para localizar o verbo transitivo, olhamos para o núcleo sintático do objeto direto. Uma frase pode incluir mais de um verbo, por isso temos cuidado ao processar os verbos. Aqui está o código:

In [111]:
import spacy
nlp = spacy.load("en_core_web_md")
doc = nlp("find a flight from washington to sf")
for token in doc:
    if token.dep_ == "dobj":
	    print(token.head.text + token.text.capitalize())

findFlight


Neste segmento de código, aplica-se o seguinte:
* Aplicamos o pipeline à nossa frase de exemplo.
* Em seguida, identificamos o objeto direto procurando um token cujo rótulo de dependência é **dobj**.
* Quando localizamos um objeto direto, identificamos o verbo transitivo correspondente obtendo o núcleo sintático do objeto direto.
* Por fim, imprimimos o verbo e o objeto para gerar o nome dessa *intent*.

Excelente! O reconhecimento de intenção foi bem-sucedido! Aqui, reconhecemos uma única intenção. Alguns enunciados podem ter múltiplas intenções. Na próxima seção, aprenderemos a reconhecer várias intenções com base nas técnicas desta seção.

### Extraindo várias intenções com relação de conjunção

Alguns enunciados carregam múltiplas intenções. Por exemplo, considere o seguinte enunciado do corpus:
~~~python
show all flights and fares from denver to san francisco
~~~
Aqui, o usuário deseja listar todos os voos e, ao mesmo tempo, deseja ver as informações da tarifa. Uma maneira de processar é considerar essas intenções como uma intenção única e complexa. Nesse caso, podemos expressar essa intenção complexa como **action: show**, **objects: flights, fares**. Outra maneira mais comum de processar esse tipo de enunciado é rotular o enunciado com múltiplas **intenções**. No conjunto de dados, este enunciado de exemplo é marcado com duas intenções como **atis_flight#atis_airfare**:

In [112]:
doc = nlp('show all flights and fares from denver to san francisco')
displacy.render(doc, style='dep')

No diagrama anterior, vemos que o arco **dobj** conecta **show** e **flights**. O arco **conj** conecta **flights** e **fares** para indicar a relação de conjunção entre eles. A relação de conjunção é construída por uma conjunção como **e** ou **ou** e indica que um substantivo está ligado a outro substantivo por essa conjunção. Nessa situação, extraímos o objeto direto e seus conjuntos. Vamos agora ver como podemos transformar esse processo em código:

In [113]:
import spacy
nlp = spacy.load("en_core_web_md")
doc = nlp("show all flights and fares from denver to san francisco")
for token in doc:
	if token.dep_ == "dobj":
		dobj = token.text
		conj = [t.text for t in token.conjuncts]
		verb = token.head
print(verb, dobj, conj)

show flights ['fares']


Aqui, percorremos todos os tokens para localizar o objeto direto da sentença. Quando encontramos o objeto direto, obtivemos seus conjuntos. Depois disso, encontrar o verbo transitivo é o mesmo que fizemos no segmento de código anterior, extraímos a cabeça do objeto direto. Depois de extrair o verbo e dois objetos, se quisermos, podemos combinar os dois para criar dois nomes de intent – **showFlights** e **showFares**.

### Reconhecendo a intenção usando listas de palavras

Em alguns casos, outros tokens além do verbo transitivo e do objeto direto contêm a semântica da intenção do usuário. Nesse caso, você precisa ir mais fundo nas relações sintáticas e explorar mais a fundo a estrutura da frase. Como exemplo, considere o seguinte enunciado de nosso conjunto de dados:
`i want to make a reservation for a flight`

Nesta frase, o par **verbo-objeto** que melhor descreve a intenção do usuário é **want-flight**. No entanto, se observarmos a árvore de análise na Figura 6.7, veremos que **want** e **flight** não estão diretamente relacionados na árvore de análise. **want** está relacionado ao verbo transitivo **make**, e **flight** está relacionado à **reserva** de objeto direto, respectivamente:

In [114]:
doc = nlp('i want to make a reservation for a flight')
displacy.render(doc, style='dep')

O que faremos então? Podemos fazer um truque e manter uma lista de verbos auxiliares como **would like**, **want**, **make**, e **need**. Aqui está o código:

In [115]:
doc = nlp("i want to make a reservation for a flight")
dObj =None
tVerb = None
# Extract the direct object and its transitive verb
for token in doc:
    if token.dep_ == "dobj":
	    dObj = token
	    tVerb = token.head
# Extract the helper verb
intentVerb = None
verbList = ["want", "like", "need", "order"]
if tVerb.text in verbList:
    intentVerb = tVerb
else:
    if tVerb.head.dep_ == "ROOT":
        intentVerb = tVerb.head
# Extract the object of the intent
intentObj = None
objList = ["flight", "meal", "booking"]
if dObj.text in objList:
    intentObj = dObj
else:
    for child in dObj.children:
        if child.dep_ == "prep":
            intentObj = list(child.children)[0]
            break
        elif child.dep_ == "compound":
            intentObj = child
            break

print(intentVerb.text + intentObj.text.capitalize())

wantFlight


Veja o que fizemos passo a passo:
1. Começamos localizando o objeto direto e seu verbo transitivo.
2. Assim que encontramos, comparamos com nossas listas predefinidas de palavras. Para este exemplo, usamos duas listas abreviadas, **verbList** contém uma lista de verbos auxiliares e **objList** contém uma lista das possíveis palavras-objeto que queremos extrair.
3. Verificamos o verbo transitivo. Se não estiver na lista de verbos auxiliares, verificamos o verbo principal da frase (marcado por **ROOT**), que é o núcleo do verbo transitivo. Se o verbo transitivo é o verbo principal da sentença, então o núcleo sintático deste verbo é ele mesmo (**tVerb.head** é **tVerb**). Portanto, a linha **if tVerb.head.dep_ == "ROOT"** é avaliada como **True** e essa implementação funciona.
4. Em seguida, verificamos o objeto direto. Se não estiver na lista de objetos possíveis, verificamos seus filhos sintáticos. Para cada filho, verificamos se o filho é uma preposição do objeto direto. Se sim, pegamos o filho da criança (pode ter apenas um filho).
5. Por fim, imprimimos a *string** que representa o nome da *intenção*, que é **wantFlight**.

Neste momento, respire fundo. Leva tempo para digerir e processar informações, especialmente quando se trata de sintaxe de sentença. Você pode tentar frases diferentes do corpus e ver o que o script faz marcando/colocando impressões no código.

Na próxima seção, exploraremos uma ferramenta muito útil, usando listas de sinônimos. Vamos avançar para a próxima seção e aprender como tirar o melhor proveito da similaridade semântica.

## Métodos de similaridade semântica para análise semântica
A linguagem natural nos permite expressar o mesmo conceito de maneiras diferentes e com palavras diferentes. Toda língua tem sinônimos e palavras semanticamente relacionadas. Como desenvolvedor de NLP, ao desenvolver um analisador semântico para um aplicativo de *chatbot*, classificação de texto ou qualquer outro aplicativo semântico, você deve ter em mente que os usuários usam um conjunto bastante amplo de frases e expressões para cada intenção. Na verdade, se você estiver criando um *chatbot* usando uma plataforma como RASA (https://rasa.com/) ou em uma plataforma como Dialogflow (https://dialogflow.cloud.google.com/), você é solicitado a forneçer o máximo de exemplos de enunciado que você puder fornecer para cada *intent*. Em seguida, esses enunciados são usados para treinar o classificador de intenção nos bastidores.

Geralmente, há duas maneiras de reconhecer a semelhança semântica, com um dicionário de sinônimos ou com métodos de semelhança semântica baseados em vetores de palavras. Nesta seção, discutiremos as duas abordagens. Vamos começar com como usar um dicionário de sinônimos para detectar similaridade semântica.

### Usando listas de sinônimos para similaridade semântica

Já analisamos nosso conjunto de dados e vimos que verbos diferentes são usados para expressar as mesmas ações. Por exemplo, **landing**, **arriving** e **flying to** para verbos que carregam o mesmo significado, enquanto **leaving**, **departing** e **flying from** são verbos que formam outro grupo semântico.

Já vimos que, na maioria dos casos, os verbos transitivos e os objetos diretos expressam a intenção. Uma maneira fácil de determinar se dois enunciados representam a mesma intenção é verificar se os verbos e os objetos diretos são sinônimos. Vamos dar um exemplo e comparar dois exemplos de enunciados do conjunto de dados. Primeiro, preparamos um pequeno dicionário de sinônimos. Incluímos apenas as formas básicas dos verbos e substantivos. Ao fazer a comparação, também usamos a forma básica das palavras:

In [116]:
verbSynsets = [
    ("show", "list"),
    ("book", "make a reservation", "buy", "reserve")
]
objSynsets = [
    ("meal", "food"),
    ("aircraft", "airplane", "plane")
]

Cada **conjunto de sinônimos (synset)** inclui o conjunto de sinônimos do nosso domínio. Geralmente incluímos os sinônimos gerais da linguagem (airplane - plane) e os sinônimos específicos do domínio (book-buy).

Os synsets estão prontos para uso e estamos prontos para passar para o código *spaCy*. Vamos passo a passo:
1. Primeiro, construímos dois objetos **doc** correspondentes aos dois enunciados que queremos comparar, **doc** e **doc2**:

In [117]:
doc = nlp("show me all aircrafts that cp uses")
doc2 = nlp("list all meals on my flight")

2. Em seguida, extraímos o verbo transitivo e o objeto direto do primeiro enunciado:

In [118]:
for token in doc:
    if token.dep_ == "dobj":
	    obj = token.lemma_
	    verb = token.head.lemma_
	    break

3. Em seguida, fazemos o mesmo para o segundo enunciado:

In [119]:
for token in doc2:
	if token.dep_ == "dobj":
		obj2 = token.lemma_
		verb2 = token.head.lemma_
		break

In [120]:
print(f"Verb 1: {verb} - Object 1: {obj}")
print(f"\nVerb 2: {verb2} - Object 2: {obj2}")

Verb 1: show - Object 1: aircraft

Verb 2: list - Object 2: meal


4. Obtivemos um synset do primeiro verbo mostrado. Em seguida, verificamos se a segunda lista de verbos está neste synset, que retorna **True**:

In [121]:
vsyn = [syn for syn in verbSynsets if verb in syn[0]]
vsyn[0]

('show', 'list')

In [122]:
verb2 in vsyn[0]

True

5. Da mesma forma, obtemos o synset do primeiro objeto direto – aircraft. Em seguida, verificamos se a segunda refeição de objeto direto está *neste synset*, o que obviamente *não é verdade*:

In [123]:
osyn = [syn for syn in objSynsets if obj in syn[0]]
osyn[0]

('aircraft', 'airplane', 'plane')

In [124]:
obj2 in vsyn[0]

False

6. Deduzimos que os dois enunciados anteriores não se referem à mesma intenção.

As listas de sinônimos são ótimas para cálculos de similaridade semântica e muitos aplicativos de NLP do mundo real se beneficiam dessas listas pré-compiladas. O uso de sinônimos nem sempre é aplicável. Fazer um dicionário procurar cada palavra em uma frase pode se tornar ineficiente para grandes synsets. Na próxima seção, apresentaremos uma maneira mais eficiente de calcular a similaridade semântica com vetores de palavras.

### Usando vetores de palavras para reconhecer similaridade semântica
Já vimos que o vetor de palavras carrega semântica, incluindo informações de sinonímia. As listas de sinônimos são úteis se você trabalha em um domínio muito específico e o número de sinônimos é bastante baixo. Trabalhar com grandes synsets pode se tornar ineficiente em algum momento porque temos que fazer um dicionário procurar os verbos e objetos diretos a cada vez. No entanto, os vetores de palavras nos oferecem uma maneira muito conveniente e baseada em vetores para calcular a similaridade semântica.

Vamos revisar o código da subseção anterior novamente. Desta vez, vamos calcular a distância semântica entre palavras com vetores de palavras spaCy. Vamos passo a passo:
1. Primeiro, construímos dois objetos **doc** que queremos comparar:


In [125]:
doc = nlp("show me all aircrafts that cp uses")
doc2 = nlp("list all meals on my flight")

2. Em seguida, extraímos o verbo e o objeto da primeira frase:

In [126]:
for token in doc:
    if token.dep_ == "dobj":
	    obj = token
	    verb = token.head
	    break

3. Repetimos o mesmo procedimento na segunda frase:

In [127]:
for token in doc2:
    if token.dep_ == "dobj":
	    obj2 = token
	    verb2 = token.head
	    break

In [128]:
print(f"Verb 1: {verb} - Object 1: {obj}")
print(f"\nVerb 2: {verb2} - Object 2: {obj2}")

Verb 1: show - Object 1: aircrafts

Verb 2: list - Object 2: meals


4. Agora, calculamos a similaridade semântica entre dois objetos diretos usando o método de similaridade baseado em vetor de palavras de spaCy:

In [129]:
obj.similarity(obj2)
# Uma pontuação muito baixa, podemos deduzir que esses 2 enunciados não estão relacionados neste momento.

0.12760281562805176

5. Por fim, calculamos a semelhança entre os verbos:

In [130]:
verb.similarity(verb2)

0.07597791403532028

O código anterior é diferente do código anterior. Desta vez, usamos os objetos token diretamente; nenhuma extração de *lemma* é necessária. Em seguida, chamamos o método **token.similarity(token2)** de *spaCy* para calcular a distância semântica entre os objetos diretos. A pontuação resultante é muito baixa. Neste ponto, deduzimos que esses dois enunciados não representam a mesma intenção.

Esta é uma maneira fácil e eficiente de calcular a similaridade semântica. Observamos no primeiro capítulo que o spaCy fornece ferramentas fáceis de usar e eficientes para desenvolvedores de NLP, e agora podemos ver o porquê.
## Juntando tudo
Já extraímos as entidades e reconhecemos a intenção de várias maneiras. Agora estamos prontos para juntar tudo para calcular uma representação semântica para um enunciado do usuário!
1. Processaremos a declaração do conjunto de dados de exemplo:

`show me flights from denver to philadelphia on tuesday`

Manteremos um objeto de dicionário para armazenar o resultado. O resultado incluirá as entidades e a intenção.

2. Vamos extrair as entidades:

In [131]:
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_md")
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "ADP"}, {"ENT_TYPE": "GPE"}]
matcher.add("prepositionLocation", [pattern])
# Location entities
doc = nlp("show me flights from denver to philadelphia on tuesday")
matches = matcher(doc)
for mid, start, end in matches:
    print(doc[start:end])

from denver
to philadelphia


In [132]:
# Todas as entidades:
ents = doc.ents
ents

(denver, philadelphia, tuesday)

3. Com essas informações, podemos gerar a seguinte representação semântica:

In [136]:
{
    'utterance': 'show me flights from denver to philadelphia on tuesday',
	'entities': {
		'date': 'tuesday',
		'locations': {
			'from': 'denver',
			'to': 'philadelphia'
		}
	}
}

{'utterance': 'show me flights from denver to philadelphia on tuesday',
 'entities': {'date': 'tuesday',
  'locations': {'from': 'denver', 'to': 'philadelphia'}}}

4. Em seguida, realizaremos o reconhecimento de intenção para gerar uma análise semântica completa:

In [134]:
import spacy
nlp = spacy.load("en_core_web_md")
doc = nlp("show me flights from denver to philadelphia on tuesday")
for token in doc:
	if token.dep_ == "dobj":
		print(token.head.lemma_ + token.lemma_.capitalize())

showFlight


5. Depois de determinar a intenção, nossa análise semântica para esse enunciado agora se parece com isso:

In [135]:
{
    'utterance': 'show me flights from denver to philadelphia on tuesday',
	'intent ': ' showFlight',
	'entities': {
		'date': 'tuesday',
		'locations': {
			'from': 'denver',
			'to': 'philadelphia'
		}
	}
}

{'utterance': 'show me flights from denver to philadelphia on tuesday',
 'intent ': ' showFlight',
 'entities': {'date': 'tuesday',
  'locations': {'from': 'denver', 'to': 'philadelphia'}}}

O resultado final é que a representação semântica completa desse enunciado, intenção e entidades é extraída. Esta é uma saída legível por máquina e utilizável. Passamos esse resultado para o componente do sistema que fez a chamada ao aplicativo NLP para gerar uma ação de resposta.